In [1]:
!uvicorn app:app --reload

^C


In [11]:
from geopy.geocoders import Nominatim
import pandas as pd

In [20]:
class parameters:
    Model: str = "XGBoost"
    House_Direction = "đông"
    Balcony_Direction = "đông"
    Toilets = 5
    Bedrooms= 6
    Floors= 5
    Legits= "hồng"
    Facade= 2.5
    Entrance= 2.5
    Ward = "Tương Mai"
    District = "Hoàng Mai"
    
def get_inputs(features):
    print("Hello")
    inputs = {
            "House Direction": features.House_Direction,
            "Balcony Direction": features.Balcony_Direction,
            "Toilets":int(features.Toilets),
            "Bedrooms":int(features.Bedrooms),
            "Legits":features.Legits,
            "Floors": int(features.Floors),
            "Facade":float(features.Facade),
            "Entrance":float(features.Entrance)
        }
    inputs["Toilets"] = "10+" if inputs["Toilets"] > 10 else str(inputs["Toilets"])
    inputs["Bedrooms"] = "10+" if inputs["Bedrooms"] > 10 else str(inputs["Bedrooms"])
    inputs["Floors"] = "10+" if inputs["Floors"] > 10 else str(inputs["Floors"])
    
    legits = ""
    if "đỏ" in inputs["Legits"]:
        legits +="+đỏ"
    if "hồng" in inputs["Legits"]:
        legits += "+hồng"
    inputs["Legits"] = legits
    loc = Nominatim(user_agent="GetLoc")
    getLoc = loc.geocode(features.Ward+" "+features.District)
    if getLoc is None:
        raise HTTPException(status_code=400, 
                            detail = "Invalid Address")
    inputs["X"] = getLoc[-1][0]
    inputs["Y"] = getLoc[-1][1]
    data = pd.DataFrame({i:[inputs[i]] for i in inputs})
    data = data[['House Direction', 'Balcony Direction', 'Toilets','Bedrooms', 
           'Legits', 'Floors', 'Facade', 'Entrance', 'X', 'Y',
            ]]
    cates = ["House Direction", "Balcony Direction",  "Toilets", "Legits", "Floors", "Bedrooms"]
    for f in cates:
         data[f] = data[f].astype("category")
    return data

In [22]:
inputs = get_inputs(parameters())

Hello


In [24]:
from joblib import load

xgb = load("models/best_tree.pkl")
xgb.predict(inputs)[0]

ValueError: Feature shape mismatch, expected: 11, got 10